\begin{align*}
  - \Delta u = f \ \text{on }\ D(\omega) \\
\\
  f=
  \begin{cases}
      F_1, & \text{if}\ x_1 \leq 0, x \in D_{ref} \cap D(\omega) \\
      F_2, & \text{if}\ x_1 > 0, x \in D_{ref} \cap D(\omega)\\
      0, & \text{otherwise}
  \end{cases}\\
  F_1, F_2 \sim U([0, 1])
\end{align*}

In [8]:
from helpers import *
from helpers_rhs_random import *
from IPython.display import clear_output
import time

In [2]:
MESH_RESOLUTION_RANDOM_FIELD_CALCULATION = 4

In [3]:
# Calculate the eigenpairs
# for MESH_RESOLUTION_RANDOM_FIELD_CALCULATION = 4 it takes 1:30 minutes
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(MESH_RESOLUTION_RANDOM_FIELD_CALCULATION)

In [ ]:
# One combined sample for V
len_xi = 10 #!randomFieldV.J
mesh_resolution_NVA = 6
mesh_resolution_inverse_mapping = 4
mesh_resolution_solution = 4


NVA = non_varying_area(len_xi, randomFieldV)
mesh_NVA = mshr.generate_mesh(NVA, mesh_resolution_NVA)
V = fe.FunctionSpace(mesh_NVA, 'P', 1)
dofmap = V.dofmap()
dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, mesh_NVA.geometry().dim())) # Those are all the point in mesh_NVA but ordered by the dof which we need to assign by the means

xi = np.random.uniform(-np.sqrt(3), np.sqrt(3), len_xi)

# Solve Poisson with random rhs
u_mean_poisson_rhs_random_P = np.zeros(len(dof_coordinates))
F_samples = np.random.random(size=2)
u_sol_poisson_rhs_random = solve_poisson_for_given_sample_rhs_random(mesh_resolution_solution, jacobianV, xi, F_samples)


for i, P_coords in enumerate(dof_coordinates):
    P = fe.Point(P_coords)
    P_hat = inverse_mapping(P, randomFieldV, xi, mesh_resolution_inverse_mapping)
    u_mean_poisson_rhs_random_P[i] = u_sol_poisson_rhs_random(P_hat)

u_mean_poisson_rhs_random = fe.Function(V)
u_mean_poisson_rhs_random.vector()[:] = u_mean_poisson_rhs_random_P

x_coords = mesh_NVA.coordinates()[:, 0]
y_coords = mesh_NVA.coordinates()[:, 1]
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]
z_values_poisson_rhs_random = []

for i in range(len(x_coords)):
    z_values_poisson_rhs_random.append(u_mean_poisson_rhs_random(x_coords[i], y_coords[i]))


grid_z = griddata((x_coords, y_coords), z_values_poisson_rhs_random, (grid_x, grid_y), method='linear')
fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale='Viridis')])
fig.update_layout(title=f'Solution of poisson equation with random rhs', autosize=True,
                scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='u(x, y)'),
                margin=dict(l=65, r=50, b=65, t=90))

fig.show()

In [ ]:
###### Section 2 ######
###### Sobol Index Estimates ######

# Inputs

# Times for double loop MC and pick freeze:
#   
mc_sample_size = 10
mesh_resolution_fem = 3
size_of_xi = 3
P = fe.Point(0.01, 0.01)

NVA = non_varying_area(size_of_xi, randomFieldV)
if not pointInNVA(P, NVA):
    raise ValueError("Point P is not in the non-varying area")

if size_of_xi > jacobianV.J:
    raise ValueError(f"size_of_xi must be less than or equal to {jacobianV.J}")


indices = [1, 2, 3] # [1, 2, 3] are all xi
double_loop_mc_closed_sobol_index_123 = double_loop_mc(mc_sample_size, mesh_resolution_fem, P, indices, randomFieldV, jacobianV, size_of_xi)

indices = [4, 5] # [4, 5] are F1 and F2
double_loop_mc_closed_sobol_index_45 = double_loop_mc(mc_sample_size, mesh_resolution_fem, P, indices, randomFieldV, jacobianV, size_of_xi)

indices = [1, 2, 3] # [1, 2, 3] are all xi
pick_freeze_closed_sobol_index_123 = pick_freeze(mc_sample_size, mesh_resolution_fem, P, indices, randomFieldV, jacobianV, size_of_xi)

indices = [4, 5] # [4, 5] are F1 and F2
pick_freeze_closed_sobol_index_45 = pick_freeze(mc_sample_size, mesh_resolution_fem, P, indices, randomFieldV, jacobianV, size_of_xi)

print(f"Double loop MC closed sobol index 123: {double_loop_mc_closed_sobol_index_123}")
print(f"Double loop MC closed sobol index 45: {double_loop_mc_closed_sobol_index_45}")
print(f"Pick freeze closed sobol index 123: {pick_freeze_closed_sobol_index_123}")
print(f"Pick freeze closed sobol index 45: {pick_freeze_closed_sobol_index_45}")

In [ ]:
# Compare sobol indices for the samples of xi

# Inputs
mc_sample_size = 100
mesh_resolution_fem = 3
size_of_xi = 3
P = fe.Point(0, 0)

NVA = non_varying_area(size_of_xi, randomFieldV)
if not pointInNVA(P, NVA):
    raise ValueError("Point P is not in the non-varying area")

if size_of_xi > jacobianV.J:
    raise ValueError(f"size_of_xi must be less than or equal to {jacobianV.J}")

s_1 = pick_freeze(mc_sample_size, mesh_resolution_fem, P, [1], randomFieldV, jacobianV, size_of_xi)
s_2 = pick_freeze(mc_sample_size, mesh_resolution_fem, P, [2], randomFieldV, jacobianV, size_of_xi)
s_3 = pick_freeze(mc_sample_size, mesh_resolution_fem, P, [3], randomFieldV, jacobianV, size_of_xi)

print(f"S_1: {s_1}, S_2: {s_2}, S_3: {s_3}")

In [ ]:
# Sobol Index estimation by rank statistics

# Inputs
mc_sample_size = 100
mesh_resolution_fem = 3
size_of_xi = 3
P = fe.Point(0, 0)

NVA = non_varying_area(size_of_xi, randomFieldV)
if not pointInNVA(P, NVA):
    raise ValueError("Point P is not in the non-varying area")

if size_of_xi > jacobianV.J:
    raise ValueError(f"size_of_xi must be less than or equal to {jacobianV.J}")

index = 1
s_1 = rank_statistics(mc_sample_size, mesh_resolution_fem, P, index, randomFieldV, jacobianV, size_of_xi)

index = 4
s_4 = rank_statistics(mc_sample_size, mesh_resolution_fem, P, index, randomFieldV, jacobianV, size_of_xi)

print(f"Rank statistics estimates: S_1: {s_1}, S_4: {s_4}")

In [ ]:
# Compare sobol indices for the samples of xi

# Inputs
mc_sample_size_double_loop = 20
mc_sample_size_pick_freeze = 150
mc_sample_size_rank_statistics = 600
mesh_resolution_fem = 3
size_of_xi = 3
P = fe.Point(0, 0)

NVA = non_varying_area(size_of_xi, randomFieldV)
if not pointInNVA(P, NVA):
    raise ValueError("Point P is not in the non-varying area")

if size_of_xi > jacobianV.J:
    raise ValueError(f"size_of_xi must be less than or equal to {jacobianV.J}")

time_start = time.time()
s_1_double_loop_mc = double_loop_mc(mc_sample_size_double_loop, mesh_resolution_fem, P, [1], randomFieldV, jacobianV, size_of_xi)
time_after_double_loop_mc = time.time()
s_1_pick_freeze = pick_freeze(mc_sample_size_pick_freeze, mesh_resolution_fem, P, [1], randomFieldV, jacobianV, size_of_xi)
time_after_pick_freeze = time.time()
s_1_rank_statistics = rank_statistics(mc_sample_size_rank_statistics, mesh_resolution_fem, P, 1, randomFieldV, jacobianV, size_of_xi)
time_after_rank_statistics = time.time()

#! measure time and print it
clear_output(wait=True)

print(f"Double loop MC: time: {round(time_after_double_loop_mc - time_start, 1)} sec., s_1: {s_1_double_loop_mc}, \n"
    f"Pick Freeze: {round(time_after_pick_freeze - time_after_double_loop_mc, 1)} sec., s_1: {s_1_pick_freeze}, \n"
    f"Rank statistics: {round(time_after_rank_statistics - time_after_pick_freeze, 1)} sec., s_1 {s_1_rank_statistics}")


In [ ]:
mc_sample_size = 10
mesh_resolution_fem = 3
size_of_xi = 3
s1 = functional_valued_output_sobol_estimation(mc_sample_size, mesh_resolution_fem, [1], randomFieldV, jacobianV, size_of_xi)
s2 = functional_valued_output_sobol_estimation(mc_sample_size, mesh_resolution_fem, [2], randomFieldV, jacobianV, size_of_xi)

print(f"s1: {s1}, s2: {s2}")

In [ ]:
mc_sample_size = 8000 # 8000 -> 304 min.
mesh_resolution_fem = 3
size_of_xi = 3
P = fe.Point(0.01, 0.01)

S_single, S_total = compare_all_sobols_rhs_random(mc_sample_size, mesh_resolution_fem, randomFieldV, jacobianV, size_of_xi, P)

In [ ]:
# Results from 8000 samples with
# mesh_resolution_fem = 3
# size_of_xi = 3
# P = fe.Point(0.01, 0.01)
# F1, F2 ~ U(0, 1)
# S_single = np.array([0.01101213, 0.01319693, 0.00928177, 0.45107665, 0.5482512])
# S_total = np.array([0.0498094, 0.06279766, 0.050678, 0.48407903, 0.55175134])
fig, ax = plt.subplots(figsize=(10, 5))

# Set width for each bar
bar_width = 0.35

ax.bar(np.arange(len(S_single)), S_single, width=bar_width, label='First Order')
ax.bar(np.arange(len(S_single)) + bar_width, S_total, width=bar_width, label='Total Effect')
x_labels = np.concatenate(([fr"$\xi_{i+1}$" for i in range(size_of_xi)], [r"$F_1$", r"$F_2$"]))
ax.set_xticklabels(x_labels)
ax.set_xticks(np.arange(len(S_single)) + bar_width / 2)
ax.set_ylabel('Sensitivity [-]')
ax.set_title(f'Sample Size: {mc_sample_size}')
ax.grid(True)
ax.legend()
plt.show()

In [ ]:
from helpers_rhs_random import rhs_random_functional_valued_output_sobol_estimation_u_hat, poisson_rhs_random_plot_sobols

mesh_resolution = 4
mc_sample_size = 10
size_of_xi = 4
S_single, S_total = rhs_random_functional_valued_output_sobol_estimation_u_hat(mc_sample_size, mesh_resolution, size_of_xi)
poisson_rhs_random_plot_sobols(S_single, S_total, mc_sample_size, title=r"$\hat{u}$ on reference domain")

In [ ]:
from helpers_rhs_random import rhs_random_functional_valued_output_sobol_estimation_u, poisson_rhs_random_plot_sobols

mesh_resolution = 4
mc_sample_size = 10
size_of_xi = 4
S_single, S_total = rhs_random_functional_valued_output_sobol_estimation_u(mc_sample_size, mesh_resolution, size_of_xi)
poisson_rhs_random_plot_sobols(S_single, S_total, mc_sample_size, title=r"u on non-varying domain")

In [ ]:
# Results for 10.000 samples, mesh_resolution = 4, size_of_xi = 6, NVA_radius = 0.68
u_S_single = np.array([0.011128391759457677, 0.002311681484487409, 0.0011435140727115887, 0.006121995311750717, 0.002149410391514706, 0.0017625322992047578])
u_S_total = np.array([0.014490995891868107, 0.004963726653825941, 0.002027576207192274, 0.006086289757896165, 0.003542663307469733, 0.002563563831820648])

u_hat_S_single = np.array([0.0013914556683523155, 0.0006031170435941264, -1.194876361821121e-05, 0.002113369969845415, 0.0001275371344608343, 0.001994153612394076])
u_hat_S_total = np.array([0.0002502586933177696, 0.008199586466509985, 0.0005881057367373704, 0.001219927771522511, 0.0024156204035458936, -0.0005069770805945322])

poisson_plot_sobols(u_S_single, u_S_total, 10000, title=r"u on non-varying domain rhs random")
poisson_plot_sobols(u_hat_S_single, u_hat_S_total, 10000, title=r"$\hat{u}$ on reference domain")

In [ ]:
from helpers_rhs_random import poisson_rhs_random_analyse_two_resolutions_from_data_u_hat
import fenics as fe

poisson_rhs_random_analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.5, 0.5))

In [ ]:
from helpers import calculate_vector_field_eigenpairs
from helpers_rhs_random import poisson_rhs_random_sobol_calc_indices_from_data, poisson_rhs_random_plot_sobols

mesh_res = 8
size_of_xi = 6
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

S_single, S_total, mc_sample_size = poisson_rhs_random_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res,
                                                                         size_of_xi=6,
                                                                         randomFieldV=randomFieldV, jacobianV=jacobianV)
poisson_rhs_random_plot_sobols(S_single, S_total, mc_sample_size, title=f"Sobol Indices for Poisson Problem rhs random Mesh resolution {mesh_res}")